In [2]:
import ultralytics
print(ultralytics.__file__)

# fine if it reports local path - Bone_Fracture_Detection\ultralytics\ultralytics\__init__.py

# there should be a right torch version - if somethimg happens we may need reinstallation

C:\Users\Admin\source\repos\Bone_Fracture_Detection\ultralytics\ultralytics\__init__.py


In [3]:
from ultralytics import YOLO
from ultralytics import settings
settings.update({"wandb": True})
import wandb


In [4]:
wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\Admin\_netrc.
wandb: Currently logged in as: gristrkv (gristrkv-klagenfurt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_config = {
    'method': 'grid'
    }

In [6]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [7]:
parameters_dict = {
    'epochs': {
        'value': 50
        },
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'lr0': {
        'values': [0.0005, 0.001, 0.002]
        },
    'lrf': {
        'values': [0.0005, 0.001, 0.002]
        },
    'batch': {
        'values': [4, 8, 16]
        },
    }

sweep_config['parameters'] = parameters_dict

In [8]:
sweep_id = wandb.sweep(sweep_config, project="bone_fracture_detection")
#sweep_id = wandb.sweep('cfg/sweep_cfg.yaml', project="bone_fracture_detection")

Create sweep with ID: uivd7dtr
Sweep URL: https://wandb.ai/gristrkv-klagenfurt-university/bone_fracture_detection/sweeps/uivd7dtr


In [9]:
import os

PROJECT_ROOT = r"C:\Users\Admin\source\repos\Bone_Fracture_Detection"
os.chdir(PROJECT_ROOT)

print("CWD:", os.getcwd())

CWD: C:\Users\Admin\source\repos\Bone_Fracture_Detection


In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
BASE_CFG = r"cfg\base_cfg.yaml"
MODEL = "yolo11n.pt"
DATA = r"data\data.yaml"


def train():
    wandb.init()
    overrides = dict(wandb.config)
    overrides["cfg"] = BASE_CFG
    overrides["model"] = MODEL
    overrides["data"] = DATA

    model = YOLO("yolo11n.pt")   # pretrained
    model.train(device="cuda:0", **overrides)
    wandb.finish()

In [ ]:
wandb.agent(sweep_id, train)